In [ ]:
# Week 2: Data Cleaning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Week 2 Environment Ready!")

# Load your dataset (Titanic or your chosen dataset)
# If using Titanic from local file:
df = pd.read_csv(r'C:\Users\khadi\Datasets\titanic_large.csv')  # or your file path

# If downloading fresh:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")

In [ ]:
print("=== DATA ASSESSMENT (BEFORE CLEANING) ===")

print("\n1. Dataset Overview:")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

print("\n2. First 10 rows:")
display(df.head(10))

print("\n3. Dataset Information:")
df.info()

print("\n4. Missing Values:")
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])

print("\n5. Duplicate Rows:")
print(f"Duplicate rows: {df.duplicated().sum()}")

print("\n6. Basic Statistics:")
display(df.describe())

In [ ]:
import matplotlib.pyplot as plt

print("=== VISUAL ASSESSMENT ===")

# Plot missing values
plt.figure(figsize=(10, 6))
missing_count = df.isnull().sum()
missing_count[missing_count > 0].plot(kind='bar')
plt.title('Missing Values Before Cleaning')
plt.ylabel('Count')
plt.show()

# Save original dataset for comparison
df_before = df.copy()
print("✅ Original dataset saved for comparison")


In [ ]:
print("=== HANDLING MISSING VALUES ===")

# Strategy 1: Remove columns with too many missing values
# If a column has more than 50% missing values, consider dropping
missing_percent = (df.isnull().sum() / len(df)) * 100
columns_to_drop = missing_percent[missing_percent > 50].index
print(f"Columns to consider dropping: {list(columns_to_drop)}")

# For Titanic dataset, Cabin has many missing values - we might drop it
if 'Cabin' in df.columns:
    df = df.drop('Cabin', axis=1)
    print("Dropped 'Cabin' column (too many missing values)")

# Strategy 2: Fill numerical missing values with median
numerical_columns = df.select_dtypes(include=[np.number]).columns
for col in numerical_columns:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)
        print(f"Filled missing values in {col} with median: {df[col].median()}")

# Strategy 3: Fill categorical missing values with mode
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
        df[col].fillna(mode_value, inplace=True)
        print(f"Filled missing values in {col} with mode: {mode_value}")

print("Missing values after cleaning:")
print(df.isnull().sum())

In [ ]:
print("=== HANDLING DUPLICATES ===")

duplicates_before = df.duplicated().sum()
print(f"Duplicate rows before: {duplicates_before}")

# Remove duplicates
df = df.drop_duplicates()

duplicates_after = df.duplicated().sum()
print(f"Duplicate rows after: {duplicates_after}")
print(f"Removed {duplicates_before - duplicates_after} duplicate rows")

In [ ]:
print("=== HANDLING OUTLIERS ===")

# Method 1: Identify outliers using IQR
def detect_outliers_iqr(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = column[(column < lower_bound) | (column > upper_bound)]
    return outliers

# Check numerical columns for outliers
numerical_columns = df.select_dtypes(include=[np.number]).columns
for col in numerical_columns:
    outliers = detect_outliers_iqr(df[col])
    if len(outliers) > 0:
        print(f"{col}: {len(outliers)} outliers detected")
        
        # Visualize outliers
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        df[col].hist(bins=30)
        plt.title(f'{col} - Distribution')
        
        plt.subplot(1, 2, 2)
        df[col].plot(kind='box')
        plt.title(f'{col} - Boxplot')
        plt.tight_layout()
        plt.show()

In [ ]:
print("=== DATA ASSESSMENT (AFTER CLEANING) ===")

print(f"Dataset shape after cleaning: {df.shape}")
print(f"Rows removed: {df_before.shape[0] - df.shape[0]}")
print(f"Columns removed: {df_before.shape[1] - df.shape[1]}")

print("\nMissing values after cleaning:")
print(df.isnull().sum())

print("\nData types:")
print(df.dtypes)

# Save cleaned dataset
df_after = df.copy()
print("✅ Cleaned dataset saved")

In [ ]:
print("📊 WEEK 2 ASSIGNMENT: BEFORE vs AFTER CLEANING REPORT")
print("=" * 60)

# Create comparison DataFrame
comparison = pd.DataFrame({
    'Metric': ['Total Rows', 'Total Columns', 'Missing Values', 'Duplicate Rows'],
    'Before Cleaning': [
        df_before.shape[0],
        df_before.shape[1],
        df_before.isnull().sum().sum(),
        df_before.duplicated().sum()
    ],
    'After Cleaning': [
        df_after.shape[0],
        df_after.shape[1],
        df_after.isnull().sum().sum(),
        df_after.duplicated().sum()
    ],
    'Improvement': [
        df_before.shape[0] - df_after.shape[0],
        df_before.shape[1] - df_after.shape[1],
        df_before.isnull().sum().sum() - df_after.isnull().sum().sum(),
        df_before.duplicated().sum() - df_after.duplicated().sum()
    ]
})

display(comparison)

print("\n🔍 KEY CLEANING ACTIONS PERFORMED:")
cleaning_actions = []

# Document what was done
if 'Cabin' in df_before.columns and 'Cabin' not in df_after.columns:
    cleaning_actions.append("Dropped 'Cabin' column (too many missing values)")

if df_before.duplicated().sum() > df_after.duplicated().sum():
    cleaning_actions.append(f"Removed {df_before.duplicated().sum() - df_after.duplicated().sum()} duplicate rows")

if df_before.isnull().sum().sum() > df_after.isnull().sum().sum():
    cleaning_actions.append(f"Handled {df_before.isnull().sum().sum() - df_after.isnull().sum().sum()} missing values")

for action in cleaning_actions:
    print(f"• {action}")

print(f"\n✅ DATA QUALITY IMPROVEMENT: {((df_before.isnull().sum().sum() - df_after.isnull().sum().sum()) / df_before.isnull().sum().sum() * 100):.1f}% reduction in missing values")

In [ ]:
# Save cleaned dataset to CSV
df_after.to_csv('titanic_cleaned.csv', index=False)

# Save the notebook
print("💾 Save this notebook as 'week2_data_cleaning.ipynb'")
print("📁 Upload to GitHub: week2_data_cleaning.ipynb + titanic_cleaned.csv")
print("🎯 Assignment 2 Complete!")